In [1]:
pip install dash dash-core-components dash-html-components pandas numpy plotly


Note: you may need to restart the kernel to use updated packages.


In [1]:
import base64
import io


In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import plotly.graph_objects as go

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Stock Price Prediction Dashboard"),
    
    dcc.Upload(
        id='upload-data',
        children=html.Button('Upload CSV'),
        multiple=False
    ),
    
    dcc.Graph(id='stock-price-graph'),
    
    html.Div(id='prediction-output')
])

# Function to preprocess the data and make predictions
def preprocess_and_predict(data):
    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[['Close']])
    
    # Create sequences for LSTM
    sequence_length = 60
    train_data_len = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_data_len]
    
    def create_sequences(data, sequence_length):
        X, y = [], []
        for i in range(sequence_length, len(data)):
            X.append(data[i-sequence_length:i, 0])
            y.append(data[i, 0])
        return np.array(X), np.array(y)

    X_train, y_train = create_sequences(train_data, sequence_length)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # Build LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=10)

    # Make predictions
    test_data = scaled_data[train_data_len - sequence_length:]
    X_test, _ = create_sequences(test_data, sequence_length)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    
    return predictions, data['Close'].values[train_data_len:]

# Callback to update the graph and output based on uploaded data
@app.callback(
    Output('stock-price-graph', 'figure'),
    Output('prediction-output', 'children'),
    Input('upload-data', 'contents')
)
def update_output(contents):
    if contents is None:
        return go.Figure(), "Upload a CSV file to see predictions."
    
    # Read uploaded CSV file
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    
    predictions, actual_prices = preprocess_and_predict(df)
    
    # Create figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Actual Price'))
    fig.add_trace(go.Scatter(x=df['Date'].iloc[-len(predictions):], y=predictions.flatten(), mode='lines', name='Predicted Price'))
    
    return fig, "Predictions made successfully."

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8090)

#  port number http://127.0.0.1:8090/
